# Seq2Seq

이름 : 남예진

기수 : 11기

작성자 : 10 기 신재우

In [1]:
# colab 환경에서 학습을 진행하실 분들은 구글드라이브를 연동해주세요
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 시작 전 실행할 것들

In [2]:
%pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 55.5 MB/s eta 0:00:00


In [3]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
import pandas as pd
import numpy as np
import tqdm
import spacy
import torchtext
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import random
import os
from ast import literal_eval
import torchtext.vocab as vocab
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

### 경로 설정

In [4]:
path_to_folder = "/content/drive/MyDrive/[0225]RNN + Transformers_YejinNam"

In [5]:
path_train = os.path.join(path_to_folder, "Dataset/train_dataset_Tokenized.csv")
path_val = os.path.join(path_to_folder, "Dataset/val_dataset_Tokenized.csv")
path_test = os.path.join(path_to_folder, "Dataset/test_dataset_Tokenized.csv")
train_dataset = pd.read_csv('/content/drive/MyDrive/[0225]RNN + Transformers_YejinNam/Dataset/train_dataset_Tokenized.csv', index_col = 0)
val_dataset = pd.read_csv('/content/drive/MyDrive/[0225]RNN + Transformers_YejinNam/Dataset/val_dataset_Tokenized.csv', index_col = 0)
test_dataset = pd.read_csv('/content/drive/MyDrive/[0225]RNN + Transformers_YejinNam/Dataset/test_dataset_Tokenized.csv', index_col = 0)

# Vocabulary

In [6]:
train_dataset['en_tokens'] = train_dataset['en_tokens'].apply(literal_eval)
train_dataset['kr_tokens'] = train_dataset['kr_tokens'].apply(literal_eval)

val_dataset['en_tokens'] = val_dataset['en_tokens'].apply(literal_eval)
val_dataset['kr_tokens'] = val_dataset['kr_tokens'].apply(literal_eval)

test_dataset['en_tokens'] = test_dataset['en_tokens'].apply(literal_eval)
test_dataset['kr_tokens'] = test_dataset['kr_tokens'].apply(literal_eval)

train_dict = train_dataset.to_dict(orient = 'records')
val_dict = val_dataset.to_dict(orient = 'records')
test_dict = test_dataset.to_dict(orient = 'records')

In [7]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"
sos_token = "<sos>"
eos_token = "<eos>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

In [8]:
def yield_tokens(data, token_field):
    for item in data:
        yield item[token_field]

en_vocab = vocab.build_vocab_from_iterator(
    yield_tokens(train_dict, "en_tokens"),
    specials=special_tokens,
    min_freq=min_freq
)

kr_vocab = vocab.build_vocab_from_iterator(
    yield_tokens(train_dict, "kr_tokens"),
    specials=special_tokens,
    min_freq=min_freq
)

en_vocab.set_default_index(en_vocab[unk_token])
kr_vocab.set_default_index(kr_vocab[unk_token])

In [9]:
vocab_size_en = len(en_vocab)
vocab_size_kr = len(kr_vocab)
print("English Vocabulary Length : ", vocab_size_en)
print("Korean Vocabulary Length : ", vocab_size_kr)

English Vocabulary Length :  9772
Korean Vocabulary Length :  10442


# 문제 2 (Vocabulary Matching)

토큰화된 단어들을 Vocabulary 과 매칭 시켜서 **벡터**로 바꿔준 뒤에 **텐서**로 바꿔주는 단계까지 포함되어야 합니다!

힌트 :

- $\texttt{en_vocab.lookup_indices}$, 혹은 $\texttt{kr_vocab.lookup_indices}$
- 한줄로 표현이 가능합니다!
- en_indices 와 kr_indices 는 리스트이며, 텐서화 된 것들을 각 토큰마다 리스트로 가져야 합니다!

In [10]:
en_indices = []
for word in train_dataset['en_tokens']:
  en_indices.append(torch.tensor(en_vocab.lookup_indices(word)))

train_dataset['en_indices'] = en_indices

kr_indices = []
for word in train_dataset['kr_tokens']:
  kr_indices.append(torch.tensor(kr_vocab.lookup_indices(word)))

train_dataset['kr_indices'] = kr_indices

In [11]:
en_indices = []
for word in val_dataset['en_tokens']:
  en_indices.append(torch.tensor(en_vocab.lookup_indices(word)))

val_dataset['en_indices'] = en_indices

kr_indices = []
for word in val_dataset['kr_tokens']:
  kr_indices.append(torch.tensor(kr_vocab.lookup_indices(word)))

val_dataset['kr_indices'] = kr_indices

In [12]:
en_indices = []
for word in test_dataset['en_tokens']:
  en_indices.append(torch.tensor(en_vocab.lookup_indices(word)))

test_dataset['en_indices'] = en_indices

kr_indices = []
for word in test_dataset['kr_tokens']:
  kr_indices.append(torch.tensor(kr_vocab.lookup_indices(word)))

test_dataset['kr_indices'] = kr_indices

# Data Loaders

In [13]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        item = self.dataframe.iloc[idx]
        en_indices = item['en_indices']
        kr_indices = item['kr_indices']
        return en_indices, kr_indices

In [14]:
batch_no = 80

def collate_fn(batch):
    en_indices, kr_indices = zip(*batch)
    en_indices_padded = pad_sequence(en_indices, batch_first=True, padding_value=0)
    kr_indices_padded = pad_sequence(kr_indices, batch_first=True, padding_value=0)

    return en_indices_padded, kr_indices_padded

train_custom_dataset = CustomDataset(train_dataset[['en_indices', 'kr_indices']])
val_custom_dataset = CustomDataset(val_dataset[['en_indices', 'kr_indices']])
test_custom_dataset = CustomDataset(test_dataset[['en_indices', 'kr_indices']])

train_loader = DataLoader(train_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)

In [15]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_indices"] for example in batch]
        batch_kr_ids = [example["kr_indices"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_kr_ids = nn.utils.rnn.pad_sequence(batch_kr_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

### 잘 매칭이 되었는지 확인해보기!

In [16]:
for i, (en_indices, kr_indices) in enumerate(train_loader):
    print(f"Batch {i+1}")
    print(f"Shape of en_indices: {en_indices.shape}")
    print(f"Shape of kr_indices: {kr_indices.shape}")

    print("Sample en_indices batch:", en_indices[:1])
    print("Sample kr_indices batch:", kr_indices[:1])

    if i == 1:
        break

Batch 1
Shape of en_indices: torch.Size([80, 29])
Shape of kr_indices: torch.Size([80, 35])
Sample en_indices batch: tensor([[   2,   10,  117, 4591,    6,  257,   75,   39,  118,    7,  118,    8,
          302,    6,  131,    4,    3,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0]])
Sample kr_indices batch: tensor([[  2, 104,  82, 198,   7,  95,   5,  11,  30, 424,  87,  18,   4,   3,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0]])
Batch 2
Shape of en_indices: torch.Size([80, 28])
Shape of kr_indices: torch.Size([80, 36])
Sample en_indices batch: tensor([[   2,   17,   23,  673,    7,  290, 1209,   19,   32,    4,    3,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]])
Sample kr_indices batch: tensor([[   2,   10,  109,   47,   50, 6381,  132,   11, 5679,    7,   12,   14,
            4,    3,    0,    0

# 문제 3 (Modules without Attention!)

기본적인 Seq2seq 모듈들은 Teacher Forcing 까지 포함해서 진행해주시길 바랍니다.

In [ ]:
class Encoder(nn.Module):
  """
  Encoder 내에서는 Input Language 의 Vocab 크기, Embedding 크기, Hidden 크기가 들어갑니다.

  Embedding 과정에서 Input 크기 --> Embedding 크기 로 진행됩니다.
  GRU 내에서는 Embedding 크기 --> Hidden 크기 로 진행됩니다.

  Input : [Batch_size, Input Sequences Max Size]
  Output : [Batch_size, Input Sequences Max Size, Hidden Size]
  Hidden : [1, Batch_size, Hidden Size]

  힌트 :
  - Batch 가 앞에 있기 때문에 GRU 는 batch_first = True 라는 파라미터를 설정해줘야 합니다.
  - GRU 에서는 Output, Hidden Vector, 총 2개를 뱉어줍니다.
  - GRU 는 nn.GRU, Embedder 는 nn.Embedding 를 사용해주면 됩니다.
  """
  def __init__(self, input_size, embedding_encoder_size, hidden_size):
    super(Encoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size, embedding_encoder_size)
    self.gru = nn.GRU(embedding_encoder_size, hidden_size, batch_first=True)

  def forward(self, input):
    embedded = self.embedding(input)
    output, hidden = self.gru(embedded)
    print(output.shape)
    return output, hidden

In [ ]:
class Decoder(nn.Module):
  """
  Decoder 내에서는 Hidden 크기, Embedding Decoder 크기 (Embedding Encoder 크기와 동일), Output Language 의 Vocab 의 크기가 들어갑니다.

  Embedding 과정에서 Output 크기 --> Embedding 크기 로 진행됩니다.
  GRU 내에서는 Embedding 크기 --> Hidden 크기 로 진행됩니다.
  Linear 내에서는 Hidden 크기 --> Output 크기 로 진행됩니다.

  Teacher Forcing 은 디코더를 구성하는 코드에서는 고려를 안해도 되며, 나중에 Seq2Seq 모듈에서 진행해주시길 바랍니다.

  Input 과 Hidden 크기의 경우 Seq2Seq 모듈 내에서 차원을 맞춰주시면 됩니다.
  """
  def __init__(self, hidden_size, embedding_decoder_size, output_size):
    super(Decoder, self).__init__()
    self.embedding = nn.Embedding(output_size, embedding_decoder_size)
    self.gru = nn.GRU(embedding_decoder_size, hidden_size, batch_first=True)
    self.fc = nn.Linear(hidden_size, output_size)
    self.output_size=output_size

  def forward(self, input, hidden):
    embedded = self.embedding(input)
    output, hidden = self.gru(embedded, hidden)
    output= output.squeeze(1) #shape : (batch_size, hidden_size)
    preds=self.fc(output)
    return preds, hidden

In [ ]:
class Seq2Seq(nn.Module):
  """
  Seq2Seq 내에서는 Encoder, Decoder, Device가 들어간다.
  Init 단계에서는 할 것 x, forward 단계에서 for loop과 if 조건문을 활용해 teacher forcing을 적용시켜 준다.

  Input : [Batch_size, Input Sequence Max Size]
  Target : [Batch_size, Target Sequence Max Size]
  Output : [Batch_size, Input Sequences Max Size, Input Language Vocab Size]
  (output은 vocabulary로 표현해야하기 때문)

  힌트:
  - outputs = 0으로 채워진 텐서. decoder의 t번째 아웃풋을 outputs의 t번째 칸에 넣으며
  for loop 진행
  - Target의 첫번째 요소 (:, 0)은 항상 <sos>!
  - random.random() < teacher_forcing_ratio를 통해 if 조건문
  - Teacher Force가 적용이 안될 때는 decoder의 아웃풋에는 모든 토큰들에 대한 점수가 나오게 된다.
  이것들 중에서 가장 최댓값만 중요하기에 argmax를 이용해서 최댓값의 index를 다음 decoder의 input으로 넣어주면 된다.
  - squeeze와 unsqueeze를 적극적으로 활용해서 차원 조정
  """
  def __init__(self, encoder, decoder, device):
    super(Seq2Seq, self).__init__()
    self.encoder=encoder
    self.decoder=decoder
    self.device=device

  def forward(self, input, target, teacher_forcing_ratio=0.5):
    batch_size=target.shape[0]
    target_len=target.shape[1]
    target_vocab_size=self.decoder.output_size

    outputs=torch.zeros(batch_size, target_len, target_vocab_size).to(self.device)
    #인코더의 마지막 hidden state를 가져와서 decoder의 첫 hidden state로 사용
    _, hidden=self.encoder(input)


    #<sos>토큰을 decoder의 첫 입력으로 사용
    decoder_input=target[:, 0].unsqueeze(1)


    for t in range(1, target_len):
      output, hidden = self.decoder(decoder_input, hidden)
      if output.shape[1] == 1:
        output = output.squeeze(1)

      outputs[:, t, :] = output
      teacher_force = random.random() < teacher_forcing_ratio
      top1 = output.argmax(1)

      decoder_input=target[:, t] if teacher_force else top1
      decoder_input = decoder_input.unsqueeze(1)

    return outputs


# Training

In [ ]:
input_dim = len(en_vocab)
output_dim = len(kr_vocab)
encoder_embedding_dim = 512
decoder_embedding_dim = 512
hidden_dim = 1024

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(input_dim, encoder_embedding_dim, hidden_dim)
decoder = Decoder(hidden_dim, decoder_embedding_dim, output_dim)
model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
model.decoder.output_size

10442

In [ ]:
model.decoder

Decoder(
  (embedding): Embedding(10442, 512)
  (gru): GRU(512, 1024, batch_first=True)
  (fc): Linear(in_features=1024, out_features=10442, bias=True)
)

### 모델 확인 + Initialize 시키기

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 32,600,266 trainable parameters


### 하이퍼 파라미터 설정!

In [ ]:
unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

n_epochs = 10
teacher_forcing_ratio = 0.5

## Training 코드!

In [ ]:
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0
    for (en_indices, kr_indices) in train_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        optimizer.zero_grad()

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio)  # Target 에서는 <eos> 는 무시하고 진행합니다.
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)

        kr_indices = kr_indices[:, 1:].contiguous().view(-1)  # Target 에서 <sos> 를 무시하기 위함입니다.

        loss = criterion(output, kr_indices)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    train_loss = epoch_loss

    model.eval()
    epoch_loss = 0
    with torch.no_grad():
      for (en_indices, kr_indices) in val_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio = 0.0)
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        kr_indices = kr_indices[:, 1:].contiguous().view(-1)

        loss = criterion(output, kr_indices)
        epoch_loss += loss.item()

    print(f'Epoch: {epoch+1}, Train Loss : {train_loss / len(train_loader)}, Val Loss: {epoch_loss / len(val_loader)}')

Epoch: 1, Train Loss : 2.8371290709177654, Val Loss: 3.029241611087133
Epoch: 2, Train Loss : 2.5208421681722006, Val Loss: 2.8872933387756348
Epoch: 3, Train Loss : 2.3398301277160645, Val Loss: 2.7794731609405035
Epoch: 4, Train Loss : 2.17648606300354, Val Loss: 2.724399483393109
Epoch: 5, Train Loss : 2.03092857615153, Val Loss: 2.771231764838809
Epoch: 6, Train Loss : 1.85759472211202, Val Loss: 2.8460091863359724
Epoch: 7, Train Loss : 1.695683040301005, Val Loss: 2.8968727626497786
Epoch: 8, Train Loss : 1.5312103741963705, Val Loss: 2.903004665223379
Epoch: 9, Train Loss : 1.406258090019226, Val Loss: 2.987535211775038
Epoch: 10, Train Loss : 1.2933436034520467, Val Loss: 3.05625637750777


파라미터 저장

In [ ]:
path_to_save = os.path.join(path_to_folder, "model_without_attention.pt")
torch.save(model.state_dict(), path_to_save)

### 적용시켜보기

In [ ]:
test_batch_en, test_batch_kr = next(iter(test_loader))
test_en = test_batch_en[0].numpy()
test_kr = test_batch_kr[0].numpy()
model.eval()
with torch.no_grad():
  output = model(test_batch_en.to(device), test_batch_kr.to(device)[:, :-1], teacher_forcing_ratio = 0.0)
  print(en_vocab.lookup_tokens(test_en))
  print(kr_vocab.lookup_tokens(test_kr))
  print(kr_vocab.lookup_tokens(output.argmax(2).cpu().numpy()[0]))

['<sos>', 'my', 'love', 'for', 'him', 'keeps', 'changing', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<sos>', '그르', 'ㄹ', '사랑', '하', '는', '내', '마음', '이', '자꾸', '변하', '어요', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '가', '이', '을', 'ㄴ', '를', '을', '어', 'ㅂ니다', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>'

\<unk\> 는 저희 Vocabulary 에는 존재하지 않는 토큰들을 의미합니다.

아무리 epoch 를 늘리거나 무엇을 해보려고 해도 완전히 좋은 결과가 나오지는 않을 것입니다. 해당 이유는 Attention 이 적용이 안되었기 때문이며, 다음과 같이 Attention 을 적용시켜서 한번 진행하면 결과가 다르다는 것을 알 수가 있습니다.

# 문제 4 (Modules WITH Attention)

만약에 메모리가 다 찼다면 해당 Run 을 Reconnect 시켜서 위의 Output 만 남긴 채로 제출하시면 됩니다.

**이렇게 진행하게 된다면 당연히 문제 3 이전의 코드들은 다시 돌려야 됩니다!!**

In [17]:
class Encoder(nn.Module):
  def __init__(self, input_size, embedding_encoder_size, hidden_size):
    super(Encoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size, embedding_encoder_size)
    self.gru = nn.GRU(embedding_encoder_size, hidden_size, batch_first=True)

  def forward(self, encoder_input):
    embedded = self.embedding(encoder_input)
    output, hidden = self.gru(embedded)
    return output, hidden

In [18]:
class Attention(nn.Module):
  """
  해당 모듈에서는 Attention Value 까지 찾는 단계입니다.

  Encoder 내에서의 Output 들의 개수마다 for loop 을 진행시켜서 합산시켜서 Attention Value 를 구하는 방법도 있지만,
  Encoder Output 들을 모두 Matrix 으로 표현해서 구하는 방법도 있습니다.

  Decoder_Outputs : [Batch_size, 1, Hidden_size]
  Encoder_Outputs : [Batch_size, Sequence Length, Hidden_size]

  힌트 :
  - Decoder_Outputs 의 차원이 위와 같이 안 나온다면 차원 조정을 위해서 squeeze, unsqueeze, transpose 등의 함수들을 활용해주면 됩니다.
  - Matrix Multiplication 을 진행할 때에 Batch 까지 곱해지는 것을 방지하기 위해서 torch.bmm(x, y) 를 활용해주면 됩니다.
  - 곱해줄 때에는 그저 x * y 를 해주면 됩니다.
  - 합산할 때에는 배치도 고려해야 함으로 torch.sum(x, dim = 1) 을 활용해주면 됩니다.
  - 해당 모듈 내에서 필요한 nn. 함수는 오로지 Softmax 밖에 없습니다.
  """

  def __init__(self, hidden_size):
    super(Attention, self).__init__()
    self.hidden_size=hidden_size
    self.softmax=nn.Softmax(dim=1)

  def forward(self, decoder_outputs, encoder_outputs):
    if decoder_outputs.dim()==2:
      decoder_outputs=decoder_outputs.unsqueeze(1)
    decoder_outputs=decoder_outputs.transpose(0,1)
    #행렬 곱셈을 위해 encoder_outputs의 차원을 전환 = [batch_size, hidden_size, seq_len]
    encoder_outputs=encoder_outputs.transpose(1,2)
    #배치 행렬 곱셈을 사용하여 attention 점수 계산
    #결과의 차원 = [batch_size, 1, seq_len]
    attention_scores=torch.bmm(decoder_outputs, encoder_outputs)

    #softmax를 사용하여 attention 가중치 계산
    attention_weights=self.softmax(attention_scores)
    #행렬곱을 위해 attention_weights의 차원을 전환
    #전환된 결과 : [batch_size, seq_len, 1]
    attention_weights=attention_weights.transpose(1,2)
    #context vector 계산 "attention weights"을 encoder의 각 state에 가중하여 모두 더한다
    print(attention_weights.shape)
    print(encoder_outputs.shape)
    context_vector=torch.bmm(encoder_outputs, attention_weights)
    #[batch, seq_len, 1] * [batch_size, hidden_size, seq_len] = [batch_size, 1, hidden_size]
    context_vector=context_vector.transpose(1,2)
    context_vector=context_vector.squeeze(1)

    return context_vector, attention_weights

In [19]:
class Decoder(nn.Module):
  """
  Attention Value 를 Attention 모듈을 통해서 구했기 때문에 Concatenate, Tangent Hyperbolic 등의 것들이 여기에서 적용이 됩니다.

  힌트 :
  - 모든 것을 진행하기 전에 앞서서 먼저 GRU 를 지나고 나서 Output 을 활용해야 합니다.
  - Attention 모듈도 사용해야 하기 때문에 해당 모듈을 현 모듈에서 정의도 해줘야 합니다.
  - Concatenate 의 경우 torch.cat((x, y), dim = 1) 형태로 사용하면 됩니다.
  - W_c 와 v_t 를 곱해줄 때 W_c 는 가중치 행렬이며, 이것은 그저 nn.Linear 형태로 표현이 가능합니다.
  """
  def __init__(self, hidden_size, embedding_decoder_size, output_size):
    super(Decoder, self).__init__()
    self.hidden_size=hidden_size
    self.embedding_decoder_size=embedding_decoder_size
    self.output_size=output_size
    self.embedding=nn.Embedding(output_size, embedding_decoder_size)
    self.attention=Attention(hidden_size)
    self.gru=nn.GRU(embedding_decoder_size, hidden_size)
    self.out=nn.Linear(hidden_size, output_size) #최종 출력을 위한 linear layer
    self.softmax=nn.Softmax(dim=1)
    self.concat=nn.Linear(hidden_size*2, hidden_size) #concatenated vector를 위한 linear layer

  def forward(self, decoder_input, hidden, encoder_outputs):
    embedded=self.embedding(decoder_input).view(1, -1, self.embedding_decoder_size)
    gru_output, hidden = self.gru(embedded, hidden)
    context_vector, attention_weights=self.attention(gru_output, encoder_outputs)
    gru_output=gru_output.squeeze(0)
    concat_input=torch.cat((gru_output, context_vector), dim=1)
    #matrix multiplication & tanh
    concat_output=torch.tanh(self.concat(concat_input))
    #Concatenated vector를 linear layer와 softmax를 통해 최종 output으로 변환
    output=self.out(concat_output)
    output=self.softmax(output)

    return output, hidden

In [20]:
class Seq2Seq(nn.Module):
  """
  이전과 Seq2Seq 부분은 크게 달라지는게 없습니다!
  단지 디코더가 Input 을 하나 더 받기 때문에 조심해주시길 바랍니다.
  """
  def __init__(self, encoder, decoder, device):
    super(Seq2Seq, self).__init__()
    self.encoder=encoder
    self.decoder=decoder
    self.device=device

  def forward(self, seq2seq_input, target, teacher_forcing_ratio = 0.5):
    batch_size=target.shape[0]
    target_len=target.shape[1]
    target_vocab_size=self.decoder.output_size

    outputs=torch.zeros(batch_size, target_len, target_vocab_size).to(self.device)
    #인코더의 마지막 hidden state를 가져와서 decoder의 첫 hidden state로 사용
    encoder_outputs, hidden= self.encoder(seq2seq_input)
    #<sos> 토큰을 decoder의 첫 입력으로 사용
    decoder_input=target[:, 0].unsqueeze(1)

    for t in range(1, target_len):
      #Attention 메커니즘을 사용하는 decoder의 forward 함수 호출
      #attention 메커니즘을 사용할 때는 encoder의 모든 출력을 디코더로 전달해야 한다
      output, hidden=self.decoder(decoder_input, hidden, encoder_outputs)
      outputs[:, t, :]=output
      teacher_force=random.random() < teacher_forcing_ratio

      #다음 입력 결정 : teacher forcing이면 실제 타깃 값을, 그렇지 않으면 이전 출력에서 가장 확률이 높은 단어를
      top1=output.argmax(1)
      decoder_input=(target[:, t] if teacher_force else top1).unsqueeze(1)

    return outputs

# Training

In [21]:
input_dim = len(en_vocab)
output_dim = len(kr_vocab)
encoder_embedding_dim = 512
decoder_embedding_dim = 512
hidden_dim = 1024

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(input_dim, encoder_embedding_dim, hidden_dim)
decoder = Decoder(hidden_dim, decoder_embedding_dim, output_dim)
model = Seq2Seq(encoder, decoder, device).to(device)
Decoder(hidden_dim, decoder_embedding_dim, output_dim)

Decoder(
  (embedding): Embedding(10442, 512)
  (attention): Attention(
    (softmax): Softmax(dim=1)
  )
  (gru): GRU(512, 1024)
  (out): Linear(in_features=1024, out_features=10442, bias=True)
  (softmax): Softmax(dim=1)
  (concat): Linear(in_features=2048, out_features=1024, bias=True)
)

In [22]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 32,600,266 trainable parameters


### 모델 확인 + Initialize 시키기

In [23]:
model.apply(init_weights)
print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 32,600,266 trainable parameters


### 하이퍼 파라미터 설정!

In [24]:
unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

n_epochs = 10
teacher_forcing_ratio = 0.5

### 학습 시키기!

In [25]:
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0
    for (en_indices, kr_indices) in train_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        optimizer.zero_grad()
        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio)  # Target 에서는 <eos> 는 무시하고 진행합니다.
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)

        kr_indices = kr_indices[:, 1:].contiguous().view(-1)  # Target 에서 <sos> 를 무시하기 위함입니다.

        loss = criterion(output, kr_indices)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    train_loss = epoch_loss
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
      for (en_indices, kr_indices) in val_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio = 0.0)
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        kr_indices = kr_indices[:, 1:].contiguous().view(-1)

        loss = criterion(output, kr_indices)
        epoch_loss += loss.item()
    print(f'Epoch: {epoch+1}, Train Loss : {train_loss / len(train_loader)}, Val Loss: {epoch_loss / len(val_loader)}')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25, 1])
torch.Size([80, 1024, 25])
torch.Size([80, 25,

### 파라미터 저장

In [26]:
path_to_save = os.path.join(path_to_folder, "model_WITH_attention.pt")
torch.save(model.state_dict(), path_to_save)

### 적용시켜보기

In [27]:
test_batch_en, test_batch_kr = next(iter(train_loader))
test_en = test_batch_en[0].numpy()
test_kr = test_batch_kr[0].numpy()
model.eval()
with torch.no_grad():
  output = model(test_batch_en.to(device), test_batch_kr.to(device)[:, :-1], teacher_forcing_ratio = 0.0)
  print(en_vocab.lookup_tokens(test_en))
  print(kr_vocab.lookup_tokens(test_kr))
  print(kr_vocab.lookup_tokens(output.argmax(2).cpu().numpy()[0]))

torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.Size([80, 1024, 33])
torch.Size([80, 33, 1])
torch.S

아마 결과가 온전히 잘 나오지는 않을 겁입니다. 해당 이유는 단순히 파라미터 개수가 적기 때문이며, 잘 나오기 위해서는 이보다 훨씬 늘려야 합니다. 원하신다면 Hidden_size, Embedding_size 를 늘려서 진행하는 것도 가능하긴 합니다!

과제 하시느라 정말로 고생하셨습니다!